In [ ]:
import xarray as xr
import numpy as np
from glob import glob
import xesmf
import os

In [ ]:
snodasdir = input("Enter the directory containing the raw SNODAS data. The raw SNODAS data should be in a file named '03jansnowprecip.nc': ").rstrip("/ ")
if not os.path.exists(snodasdir):
    raise Exception("SNODAS directory not found!")


In [ ]:
jan3snow = xr.open_dataset(f"{snodasdir}/03jansnowprecip.nc")
#In this file, Band1 is the liquid equivalent of snowfall, in tenths of mm. It's named this way because of the GDAL script that NOHRSC provides to convert their .dat files to .nc files.

In [ ]:
print(jan3snow["Band1"])

In [ ]:
gridsmade = input("Have you made grid files for the CONTROL and NLH simulations? These are just files which contain information basic information about coordinates, patch areas, and terrain height. If yes, there should be NetCDF files in the directory with the processed data for each simulation with the name 'gridconsts.nc'. Enter 'yes' or 'no': ")
if gridsmade.lower().strip() == "yes":
    gridsmade = True
elif gridsmade.lower().strip() == "no":
    gridsmade = False
else:
    raise Exception("Answer must be either 'yes' or 'no'!")
if gridsmade:
    controlgridpath = input("Enter the directory containing the gridconsts.nc file for the CONTROL simulation: ").rstrip("/ ")
    if not os.path.exists(f"{controlgridpath}/gridconsts.nc"):
        raise Exception("Control grid file not found!")
    controlgrid = xr.open_dataset(f"{controlgridpath}/gridconsts.nc")
else:
    controlpath = input("Enter the path to the directory containing processed data for the CONTROL simulation: ").rstrip("/ ")
    if not os.path.exists(f"{controlpath}/mvars-cart-2010-01-02-180000-g1.nc"):
        raise Exception("Post-processed files not found in specified directory!")
    nlhpath = input("Enter the path to the directory containing processed data for the NLH simulation: ").rstrip("/ ")
    if not os.path.exists(f"{nlhpath}/mvars-cart-2010-01-02-180000-g1.nc"):
        raise Exception("Post-processed files not found in specified directory!")
    control_file_init = xr.open_dataset(f"{controlpath}/mvars-cart-2010-01-02-180000-g1.nc")
    nlh_file_init = xr.open_dataset(f"{nlhpath}/mvars-cart-2010-01-02-180000-g1.nc")
    controlgrid = control_file_init[["PatchArea", "Topo"]]
    nlhgrid = nlh_file_init[["PatchArea", "Topo"]]
    control_file_init.close(); del control_file_init
    nlh_file_init.close(); del nlh_file_init
    controlgridpath = input("Enter the directory where you want to put the gridconsts.nc file for the CONTROL simulation: ").rstrip("/ ")
    nlhgridpath = input("Enter the directory where you want to put the gridconsts.nc file for the NLH simulation: ").rstrip("/ ")
    controlgrid.to_netcdf(f"{controlgridpath}/gridconsts.nc")
    nlhgrid.to_netcdf(f"{nlhgridpath}/gridconsts.nc")
controlgrid = xr.open_dataset("/moonbow/ascheb/les/2010/hires_control/gridconsts.nc")

In [ ]:
from time import perf_counter
st = perf_counter()
gridweights = xr.open_dataset("snodasgridweights.nc")
rgrid2 = xesmf.Regridder(jan3snow, controlgrid["Topo"], "bilinear", weights = gridweights)
jan3snowregrid = rgrid2(jan3snow["Band1"]); jan3snowregrid.name = "Snowfall"
jan3snowregrid.to_netcdf("/moonbow/ascheb/les/SNODAS/03jansnowprecip_regrid.nc")
et = perf_counter()
print(f"regridding took {et-st:.2f} seconds!")

In [ ]:
jan3snowregrid.close()
del jan3snowregrid
jan3snow.close()
del jan3snow